In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

## using RoBERTa trained on twitter sentiment analysis: 
## https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
classifier(["We are very happy to show you the 🤗 Transformers library.", 
            "Now Playing: JAI - VIP Mix by Finesu -> ***",
            "I kind of like you", 
           "french people yikes", 
           "LESGOOOOO", 
           "POGGERS we we",
           "this is WICKED",
           "Leather Daddy"])

[{'label': 'Positive', 'score': 0.985775887966156},
 {'label': 'Neutral', 'score': 0.9331138730049133},
 {'label': 'Positive', 'score': 0.667307436466217},
 {'label': 'Negative', 'score': 0.8625349402427673},
 {'label': 'Positive', 'score': 0.45673370361328125},
 {'label': 'Neutral', 'score': 0.6843876838684082},
 {'label': 'Negative', 'score': 0.7978640198707581},
 {'label': 'Neutral', 'score': 0.5491257905960083}]

In [3]:
import pandas as pd 
text_df = pd.read_csv('../data/Fact Tables/EmoteFactTable.csv')

In [4]:
text_df

,ID,Emote,Definition 1,Racism Implication?
0,1,Kappa,Kappa is a Twitch emote that is generally used...,0
1,2,LUL,The acronym LUL stands for “Lame Uncomfortable...,0
2,3,TriHard,The TriHard Twitch emote was created to be use...,1
3,4,PogChamp,The PogChamp emote is used to express surprise...,0
4,5,Pog,The PogChamp emote is used to express surprise...,0
...,...,...,...,...
63,64,3Head,"Used when someone says something cheesy, such ...",0
64,65,YEP,An insincere smile that many people use in sit...,0
65,66,Pepega,It is based on the Pepe the Frog character and...,0
66,67,KKona,The KKona Twitch emote is most commonly used a...,0


In [37]:
twitch_chats = text_df['Definition 1'].tolist()

In [8]:
twitch_chats

['Kappa is a Twitch emote that is generally used to relay sarcasm or as an “eye-roll” response to something the Twitch streamer says or does on screen. Known as a “troll” emote, Kappa is often spammed in chat when someone is sarcastic or checking to see if they have the Golden Kappa.',
 'The acronym LUL stands for “Lame Uncomfortable Laugh.” The LUL Twitch emote is used to express laughter and joy and depicts the late streamer, TotalBuscuit. It is often used on Twitch in the same way that you would use a laugh emote when messaging friends on other platforms.',
 'The TriHard Twitch emote was created to be used when something exciting is happening on the screen or to help generate hype in a channel. It is the image of the streamer Trihex smiling. Use it when you’re hyped or when the streamer does something exciting.',
 'The PogChamp emote is used to express surprise or excitement',
 'The PogChamp emote is used to express surprise or excitement',
 'The PogChamp emote is used to express su

In [6]:
list_polarity = classifier(twitch_chats)

In [11]:
list_polarity_label = []
for result in list_polarity:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
    list_polarity_label.append(result['label'].upper())
    #     list_polarity_label.append(round(result['score'], 4))

label: Neutral, with score: 0.8278
label: Positive, with score: 0.5579
label: Positive, with score: 0.9341
label: Positive, with score: 0.6078
label: Positive, with score: 0.6078
label: Positive, with score: 0.6078
label: Negative, with score: 0.5644
label: Neutral, with score: 0.8326
label: Neutral, with score: 0.8536
label: Neutral, with score: 0.6606
label: Neutral, with score: 0.7448
label: Positive, with score: 0.5979
label: Negative, with score: 0.669
label: Neutral, with score: 0.594
label: Positive, with score: 0.8056
label: Neutral, with score: 0.5911
label: Positive, with score: 0.9542
label: Negative, with score: 0.6981
label: Neutral, with score: 0.6951
label: Negative, with score: 0.6514
label: Negative, with score: 0.639
label: Positive, with score: 0.6598
label: Negative, with score: 0.6499
label: Neutral, with score: 0.4991
label: Positive, with score: 0.8706
label: Neutral, with score: 0.7582
label: Negative, with score: 0.7176
label: Neutral, with score: 0.6087
label:

In [12]:
text_df['BERT_polarity'] = list_polarity_label

In [13]:
text_df.to_csv('../data/Fact tables/EmoteFactTable_BERT.csv')

### Fine tuning

In [5]:
train_data = pd.read_csv('../data/processed/BERT_training.csv')

In [6]:
train_data

,label,text
0,1,Kappa
1,2,LUL
2,2,TriHard
3,2,PogChamp
4,2,Pog
...,...,...
63,1,3Head
64,1,YEP
65,0,Pepega
66,1,KKona


In [132]:
# !pip install datasets
from datasets import load_dataset

In [7]:
from datasets import Dataset 
bert_dataset = Dataset.from_pandas(train_data)

In [8]:
bert_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 68
})

In [49]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Reusing dataset yelp_review_full (/Users/Vaibhav_Beohar/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [9]:
from transformers import AutoTokenizer, RobertaTokenizer
# tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [10]:
def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=4)
    return tokenizer(examples["text"], max_length=4)
    

In [11]:
tokenized_datasets_v2 = bert_dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [15]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2))

In [16]:
# small_train_dataset[0]

In [12]:
small_train_dataset_v2 = tokenized_datasets_v2.shuffle(seed=42).select(range(65))
small_eval_dataset_v2 = tokenized_datasets_v2.shuffle(seed=42).select(range(3))

In [13]:
small_train_dataset_v2[0]

{'label': 2,
 'text': 'gachiGASM',
 'input_ids': [0, 571, 16940, 2],
 'attention_mask': [1, 1, 1, 1]}

In [14]:
from transformers import AutoModelForSequenceClassification

In [15]:
model = AutoModelForSequenceClassification.\
    from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [33]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [114]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits), # axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [115]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [141]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset_v2,
    eval_dataset=small_eval_dataset_v2,
    compute_metrics=compute_metrics,
)

AttributeError: 'TFRobertaForSequenceClassification' object has no attribute 'to'

In [117]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 65
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 27


IndexError: Target -1 is out of bounds.

### Training using Tensorflow

In [17]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [18]:
tf_train_dataset = small_train_dataset_v2.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset_v2.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [19]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# !pip install transformers-keras

In [21]:
from transformers.keras_callbacks import PushToHubCallback

In [22]:
push_to_hub_callback = PushToHubCallback(
    output_dir="../models/twitch-roberta-base-sentiment-latest", tokenizer=tokenizer, hub_model_id="veb/twitch-roberta-base-sentiment-latest"
)

/Users/Vaibhav_Beohar/Documents/VB_Mck_Docs/MIDS/W210/final_proj/Twitch-chat-pioneers/notebooks/../models/twitch-roberta-base-sentiment-latest is already a clone of https://huggingface.co/veb/twitch-roberta-base-sentiment-latest. Make sure you pull the latest changes with `repo.git_pull()`.


In [23]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3, callbacks=push_to_hub_callback)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, s

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Several commits (5) will be pushed upstream.


8/8 [==============================] - 32s 3s/step - loss: 1.1272 - sparse_categorical_accuracy: 0.3281 - val_loss: 1.0190 - val_sparse_categorical_accuracy: 0.3333
Epoch 2/3
8/8 [==============================] - 4s 513ms/step - loss: 1.1254 - sparse_categorical_accuracy: 0.2969 - val_loss: 1.1164 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 4s 495ms/step - loss: 1.0941 - sparse_categorical_accuracy: 0.3750 - val_loss: 1.0186 - val_sparse_categorical_accuracy: 0.3333


Several commits (6) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 32.0k/476M [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/veb/twitch-roberta-base-sentiment-latest
   959fae6..c8d9b7e  main -> main



In [138]:
for i in small_train_dataset_v2:
    print(i)

{'label': 2, 'text': 'gachiGASM', 'input_ids': [0, 571, 16940, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 1, 'text': 'KKona', 'input_ids': [0, 26228, 4488, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 2, 'text': 'Pog', 'input_ids': [0, 510, 2154, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 0, 'text': 'BabyRage', 'input_ids': [0, 30047, 500, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 0, 'text': 'BrokeBack', 'input_ids': [0, 26519, 1071, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 1, 'text': 'Keepo', 'input_ids': [0, 28407, 139, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 1, 'text': 'PauseChamp', 'input_ids': [0, 48532, 43225, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 0, 'text': 'MonkaW', 'input_ids': [0, 17312, 2348, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 1, 'text': 'VoHiYo', 'input_ids': [0, 37660, 30086, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 2, 'text': 'widepeepoHappy', 'input_ids': [0, 6445, 2379, 2], 'attention_mask': [1, 1, 1, 1]}
{'label': 0, 'text'

In [27]:
from transformers import pipeline, AutoModelForTokenClassification

tokenizer = AutoModelForTokenClassification.from_pretrained('veb/twitch-roberta-base-sentiment-latest', from_tf=True)
classifier = pipeline("sentiment-analysis", model='veb/twitch-roberta-base-sentiment-latest')

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing RobertaForTokenClassification.

All the weights of RobertaForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForTokenClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at veb/twitch-roberta-base-sentiment-latest.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [36]:
classifier(['i hate you'])

[{'label': 'Neutral', 'score': 0.38475385308265686}]

In [39]:
list_polarity = classifier(twitch_chats)
list_polarity_label = []
for t, result in zip(twitch_chats, list_polarity):
    print(f"Text: {t[:50]},  label: {result['label']}, with score: {round(result['score'], 4)}")
    list_polarity_label.append(result['label'].upper())
    #     list_polarity_label.append(round(result['score'], 4))

Text: Kappa is a Twitch emote that is generally used to ,  label: Neutral, with score: 0.3971
Text: The acronym LUL stands for “Lame Uncomfortable Lau,  label: Neutral, with score: 0.3835
Text: The TriHard Twitch emote was created to be used wh,  label: Neutral, with score: 0.3893
Text: The PogChamp emote is used to express surprise or ,  label: Neutral, with score: 0.3653
Text: The PogChamp emote is used to express surprise or ,  label: Neutral, with score: 0.3653
Text: The PogChamp emote is used to express surprise or ,  label: Neutral, with score: 0.3653
Text: MonkaS is an emote that is often used in Twitch ch,  label: Positive, with score: 0.3964
Text: VoHiYo is known by Twitch viewers as the “Weeb” (W,  label: Neutral, with score: 0.4029
Text: The emote is used as a laugh emote on Twitch.,  label: Neutral, with score: 0.3669
Text: The PepeLaugh emote is often accompanied by the ph,  label: Neutral, with score: 0.3705
Text: The 4Head emote is used in response to jokes or me,  label